# Quickstart with Ray AI Runtime

<img src="https://technical-training-assets.s3.us-west-2.amazonaws.com/Generic/ray_logo.png" width="20%" loading="lazy">

## Preliminaries

### Install libraries

In [1]:
!pip install -U ray
!pip install -U xgboost_ray

### Imports

In [2]:
import ray
from ray.air.config import ScalingConfig
from ray.data.preprocessors import MinMaxScaler
from ray.train.xgboost import XGBoostTrainer

### Initialize Ray runtime

In [3]:
ray.init()

2025-03-16 19:18:20,139	INFO worker.py:1841 -- Started a local Ray instance.


Python version:,3.11.11
Ray version:,2.43.0


## Load and prepare data with Ray Datasets

### Read Parquet file to Ray Dataset

In [4]:
dataset = ray.data.read_parquet(
    "s3://anyscale-training-data/intro-to-ray-air/nyc_taxi_2021.parquet"
)

Parquet Files Sample 0:   0%|          | 0.00/1.00 [00:00<?, ? file/s]

Returned `dataset` is [Ray Dataset](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.html#ray-data-dataset) - standard way to load and exchange data in Ray AI Runtime.

In AIR, Datasets are used extensively for data loading and transformation. They are meant as a last-mile bridge from ETL pipeline outputs to distributed applications and libraries in Ray.

### Split data into training and validation subsets

In [5]:
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

2025-03-16 19:18:32,637	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-03-16_19-18-14_958222_3663/logs/ray-data
2025-03-16 19:18:32,638	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet]


Running 0: 0.00 row [00:00, ? row/s]

- ReadParquet->SplitBlocks(147) 1: 0.00 row [00:00, ? row/s]

### Split datasets into blocks for parallel preprocessing

In [11]:
train_dataset = train_dataset.repartition(num_blocks=3)
valid_dataset = valid_dataset.repartition(num_blocks=3)

# Fit and transform the preprocessor on the training dataset
train_dataset = preprocessor.fit_transform(train_dataset)
# Transform the validation dataset using the fitted preprocessor
valid_dataset = preprocessor.transform(valid_dataset)

2025-03-16 19:22:01,671	INFO dataset.py:2787 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2025-03-16 19:22:01,678	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-03-16_19-18-14_958222_3663/logs/ray-data
2025-03-16 19:22:01,679	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> AllToAllOperator[Repartition] -> AllToAllOperator[Repartition] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]


Running 0: 0.00 row [00:00, ? row/s]

- Repartition 1: 0.00 row [00:00, ? row/s]

Split Repartition 2:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

- Repartition 3: 0.00 row [00:00, ? row/s]

...

- Aggregate 5: 0.00 row [00:00, ? row/s]

Sort Sample 6:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

Shuffle Map 7:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

Shuffle Reduce 8:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

- limit=1 9: 0.00 row [00:00, ? row/s]

`num_blocks` should be lower than number of cores in the cluster

### Define a preprocessor to normalize the columns by their range

In [7]:
preprocessor = MinMaxScaler(columns=["trip_distance", "trip_duration"])

[Preprocessors](https://docs.ray.io/en/latest/ray-air/key-concepts.html#preprocessors) are primitives that transform input data into features. They operate on Datasets, making them scalable and compatible with a variety of datasources and dataframe libraries.

Ray AI Runtime comes with a collection of built-in preprocessors, and you can also define your own with simple templates (see [Using preprocessors](https://docs.ray.io/en/latest/ray-air/preprocessors.html) for more information).

## Train the model with Ray Train

|<img src="https://technical-training-assets.s3.us-west-2.amazonaws.com/Scaling_model_training/data_parallelism.png" width="50%" loading="lazy">|
|:--|
|Ray Train provides distributed data parallel training capabilities. A large dataset is sharded across multiple worker nodes each containing a model copy. Gradients calculated on independent nodes are continuously synchronized with others to produce a final trained model.|

### Create XGBoost trainer

In [12]:
trainer = XGBoostTrainer(
    label_column="is_big_tip",
    num_boost_round=100,
    scaling_config=ScalingConfig(
        use_gpu=False,  # True for the GPU training, 1 GPU per worker
    ),
    params={
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
        "tree_method": "approx",  # use "gpu_hist" for GPU training
    },
    datasets={"train": train_dataset, "valid": valid_dataset},
    #preprocessor=preprocessor,
)

During training, `trainer` will use `num_blocks` workers, defined when repartitioning dataset.

Ray AI Runtime comes with built-in integrations with mang popular ML projects like PyTorch, Keras, LightGBM and more. Refer to the [Ray Train docs](https://docs.ray.io/en/latest/train/train.html#quick-start-to-distributed-training-with-ray-train) for more details. Optionally, read more about the Ray-XGBoost integration in the [Introducing Distributed XGBoost Training with Ray](https://www.anyscale.com/blog/distributed-xgboost-training-with-ray) blog post.

### Invoke training - this is computationally intensive operation

In [13]:
result = trainer.fit()

2025-03-16 19:23:06,132	WARNING callback.py:136 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`



View detailed results here: /root/ray_results/XGBoostTrainer_2025-03-16_19-23-06

Training started without custom configuration.


(XGBoostTrainer pid=6964) Started distributed worker processes: 
(XGBoostTrainer pid=6964) - (node_id=7b53d495479f92d3f5f9a6990397e19533a4aeb55c5bb3fcb36baba6, ip=172.28.0.12, pid=7017) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=7017) [19:23:15] Task [xgboost.ray-rank=00000000]:81d9a5f0c4dbca754f81694d01000000 got rank 0
(SplitCoordinator pid=7060) Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-03-16_19-18-14_958222_3663/logs/ray-data
(SplitCoordinator pid=7060) Execution plan of Dataset: InputDataBuffer[Input] -> AllToAllOperator[Repartition] -> AllToAllOperator[Repartition] -> TaskPoolMapOperator[MinMaxScaler] -> OutputSplitter[split(1, equal=True)]


(pid=7060) Running 0: 0.00 row [00:00, ? row/s]

(pid=7060) - Repartition 1: 0.00 row [00:00, ? row/s]

...


Training finished iteration 99 at 2025-03-16 19:25:56. Total running time: 2min 50s
+-------------------------------+
| Training result               |
+-------------------------------+
| checkpoint_dir_name           |
| time_this_iter_s      1.26996 |
| time_total_s          165.943 |
| training_iteration         99 |
| train-error           0.38474 |
| train-logloss         0.65462 |
| valid-error           0.38822 |
| valid-logloss         0.65822 |
+-------------------------------+


(XGBoostTrainer pid=6964) [19:25:56] [98]	train-logloss:0.65462	train-error:0.38474	valid-logloss:0.65822	valid-error:0.38822



Training finished iteration 100 at 2025-03-16 19:25:58. Total running time: 2min 51s
+-------------------------------+
| Training result               |
+-------------------------------+
| checkpoint_dir_name           |
| time_this_iter_s      1.25104 |
| time_total_s          167.194 |
| training_iteration        100 |
| train-error           0.38464 |
| train-logloss         0.65458 |
| valid-error           0.38821 |
| valid-logloss         0.65822 |
+-------------------------------+

Training finished iteration 101 at 2025-03-16 19:25:58. Total running time: 2min 51s
+-----------------------------------------+
| Training result                         |
+-----------------------------------------+
| checkpoint_dir_name   checkpoint_000000 |
| time_this_iter_s                0.00364 |
| time_total_s                  167.19743 |
| training_iteration                  101 |
| train-error                     0.38464 |
| train-logloss                   0.65458 |
| valid-error           

(XGBoostTrainer pid=6964) [19:25:57] [99]	train-logloss:0.65458	train-error:0.38464	valid-logloss:0.65822	valid-error:0.38821
(RayTrainWorker pid=7017) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/XGBoostTrainer_2025-03-16_19-23-06/XGBoostTrainer_16551_00000_0_2025-03-16_19-23-06/checkpoint_000000)
2025-03-16 19:25:59,305	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/XGBoostTrainer_2025-03-16_19-23-06' in 0.0302s.



Training completed after 101 iterations at 2025-03-16 19:25:59. Total running time: 2min 53s



The resulting object grants access to metrics, checkpoints, and errors

### Report results

In [10]:
print(f"train acc = {1 - result.metrics['train-error']:.4f}")
print(f"valid acc = {1 - result.metrics['valid-error']:.4f}")
print(f"iteration = {result.metrics['training_iteration']}")

NameError: name 'result' is not defined

## Shutdown Ray runtime

In [ ]:
ray.shutdown()

Disconnect the worker and terminate processes started by `ray.init()`.

# Connect with the Ray community

You can learn and get more involved with the Ray community of developers and researchers:

* [**Ray documentation**](https://docs.ray.io/en/latest)

* [**Official Ray site**](https://www.ray.io/)  
Browse the ecosystem and use this site as a hub to get the information that you need to get going and building with Ray.

* [**Join the community on Slack**](https://forms.gle/9TSdDYUgxYs8SA9e8)  
Find friends to discuss your new learnings in our Slack space.

* [**Use the discussion board**](https://discuss.ray.io/)  
Ask questions, follow topics, and view announcements on this community forum.

* [**Join a meetup group**](https://www.meetup.com/Bay-Area-Ray-Meetup/)  
Tune in on meet-ups to listen to compelling talks, get to know other users, and meet the team behind Ray.

* [**Open an issue**](https://github.com/ray-project/ray/issues/new/choose)  
Ray is constantly evolving to improve developer experience. Submit feature requests, bug-reports, and get help via GitHub issues.

* [**Become a Ray contributor**](https://docs.ray.io/en/latest/ray-contribute/getting-involved.html)  
We welcome community contributions to improve our documentation and Ray framework.

<img src="https://technical-training-assets.s3.us-west-2.amazonaws.com/Generic/ray_logo.png" width="20%" loading="lazy">